## Import Requird Libraries

In [1]:
from transformers import (
    T5ForConditionalGeneration,
    MT5ForConditionalGeneration,
    ByT5Tokenizer,
    PreTrainedTokenizer,
    T5TokenizerFast as T5Tokenizer,
    MT5TokenizerFast as MT5Tokenizer,
    T5Model,
    AutoModelForSeq2SeqLM,
    AutoTokenizer
)
import pandas as pd

In [2]:
import torch
import json
from tqdm import tqdm
import torch.nn as nn
from torch.optim import Adam
import nltk
import spacy
import string
import evaluate  # Bleu
from torch.utils.data import Dataset, DataLoader, RandomSampler
import pandas as pd
import numpy as np
import transformers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration, T5TokenizerFast

import warnings


In [116]:
!pip install matplotlib

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.2/9.2 MB 66.9 MB/s eta 0:00:0000:0100:01
  Using cached kiwisolver-1.4.5-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.2 MB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 104.6 MB/s eta 0:00:0000:01
  Using cached contourpy-1.1.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (301 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 101.4 MB/s eta 0:00:0000:01

[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip install --upgrade pip


##  Load  Flan-T5-Small Model

In [3]:
t5_model=AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small", return_dict=True)

##  Load  Flan-T5-Small Tokenizer

In [4]:
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")


## Article Sumarization

In [6]:
from transformers import pipeline
summarizer = pipeline("summarization", model=t5_model,tokenizer=tokenizer)
summarizer(input_text)

[{'summary_text': 'APJ abdul Kalam was born in a poor Tamil Muslim family on 15th of October in 1931 at Rameshwaram, Ramnad district of Madras presidency under British India'}]

In [5]:
input_text = "summarize: The full name of Dr. APJ abdul Kalam was Avul Pakir Jainulabdeen Abdul Kalam. He is popularly known as the Missile Man of India and People’s President. Kalam was born in a poor Tamil Muslim family on 15th of October in 1931 at Rameshwaram, Ramnad district of Madras presidency under British India (currently in Ramanathapuram District, Tamil Nadu). He was a great scientist who also served the country as the 11th President of India from 2002 to 2007. After completing his term of presidency, he returned to the civilian life of writing, education, and public service. He worked at various chief positions at ISRO and DRDO then became a Principal Scientific Adviser to the Government Of India as a Cabinet Minister.Kalam has been honored with the honorary doctorates by at least 30 universities as well as three highest civilian awards of the country (Padma Bhushan 1981, Padma Vibhushan 1990 and Bharat Ratna 1997). He was a great personality and inspiration to the youngsters of country who took his last breath at IIM, Meghalaya on 27th of July in 2015 because of the sudden cardiac arrest. Kalam is not present among us physically however his great works and contributions would be with us forever. He has mentioned his dream of making India a developed country in his book “India 2020-A vision for the New Millennium. "
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = t5_model.generate(input_ids)
for i in outputs:
    print(tokenizer.decode(i))



<pad> APJ abdul Kalam was born in a poor Tamil Muslim family on 15th


## Question Answering  

In [7]:
from transformers import pipeline
input_text="question-answering: What is sum of one and one ?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = t5_model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

<pad> one and one is the sum of two and one, one and two, and one and a half, two and three, three and one</s>


### Translation Task From English to French

In [8]:
from transformers import pipeline
input_text="english to french: My screen is blank ?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = t5_model.generate(input_ids)
print(tokenizer.decode(outputs[0]))

<pad> M'écran ne s'agit pas de l'écran. et il n'y a pas d'un écran.</s>


### Layer Name and Length

In [179]:
for name, param in t5_model.named_parameters():
    print(name, ": Shape" + " "+str(len(param) ))


shared.weight : Shape 32128
encoder.block.0.layer.0.SelfAttention.q.weight : Shape 384
encoder.block.0.layer.0.SelfAttention.k.weight : Shape 384
encoder.block.0.layer.0.SelfAttention.v.weight : Shape 384
encoder.block.0.layer.0.SelfAttention.o.weight : Shape 512
encoder.block.0.layer.0.SelfAttention.relative_attention_bias.weight : Shape 32
encoder.block.0.layer.0.layer_norm.weight : Shape 512
encoder.block.0.layer.1.DenseReluDense.wi_0.weight : Shape 1024
encoder.block.0.layer.1.DenseReluDense.wi_1.weight : Shape 1024
encoder.block.0.layer.1.DenseReluDense.wo.weight : Shape 512
encoder.block.0.layer.1.layer_norm.weight : Shape 512
encoder.block.1.layer.0.SelfAttention.q.weight : Shape 384
encoder.block.1.layer.0.SelfAttention.k.weight : Shape 384
encoder.block.1.layer.0.SelfAttention.v.weight : Shape 384
encoder.block.1.layer.0.SelfAttention.o.weight : Shape 512
encoder.block.1.layer.0.layer_norm.weight : Shape 512
encoder.block.1.layer.1.DenseReluDense.wi_0.weight : Shape 1024
encod

### Total Number of Parameters

In [180]:
sum(p.numel() for p in t5_model.parameters())

76961152

### Class for assigning final decoder layer to zero

In [82]:
import torch.nn as nn
import copy

class T5Model_Ov(nn.Module):
    _keys_to_ignore_on_load_unexpected = [
        "decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight",
    ]
    _tied_weights_keys = ["encoder.embed_tokens.weight", "decoder.embed_tokens.weight"]

    def __init__(self):#, config: t5_model.config):
        super().__init__()
        
        self.shared = nn.Embedding(t5_model_seq.config.vocab_size, t5_model_seq.config.d_model)
        self.base_model = T5Model.from_pretrained("google/flan-t5-small")
        encoder_config = copy.deepcopy(t5_model_seq.config)
        self.encoder =self.base_model.encoder# self.base_model(encoder_config, self.shared)
        self.encoder.is_decoder = False
        self.encoder.use_cache = False
        self.encoder.is_encoder_decoder = False
            
        #decoder_config = copy.deepcopy(config)
        
        self.decoder = self.base_model.decoder#self.base_model(decoder_config, self.shared)
        
        self.decoder.num_layers = self.decoder.config.num_decoder_layers
        
        self.decoder.is_decoder = True
        self.decoder.is_encoder_decoder = False
           
        # Initialize weights and apply final processing
        #self.post_init()

        # Model parallel
        self.model_parallel = False
        self.device_map = None

    def get_encoder(self):
        return self.encoder

    def get_decoder(self):
        return self.decoder

    def _prune_heads(self, heads_to_prune):
      
        for layer, heads in heads_to_prune.items():
            self.encoder.layer[layer].attention.prune_heads(heads)

#    @replace_return_docstrings(output_type=Seq2SeqModelOutput, config_class=_CONFIG_FOR_DOC)
    def forward(self ):# -> Union[Tuple[torch.FloatTensor], Seq2SeqModelOutput]:
       
        use_cache = use_cache if use_cache is not None else self.config.use_cache
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        # FutureWarning: head_mask was separated into two input args - head_mask, decoder_head_mask
        if head_mask is not None and decoder_head_mask is None:
            if self.config.num_layers == self.config.num_decoder_layers:
                warnings.warn(__HEAD_MASK_WARNING_MSG, FutureWarning)
                decoder_head_mask = head_mask

        # Encode if needed (training, first prediction pass)
        if encoder_outputs is None:
            encoder_outputs = self.encoder(
                input_ids=input_ids,
                attention_mask=attention_mask,
                inputs_embeds=inputs_embeds,
                head_mask=head_mask,
                output_attentions=output_attentions,
                output_hidden_states=output_hidden_states,
                return_dict=return_dict,
            )
        elif return_dict and not isinstance(encoder_outputs, BaseModelOutput):
            encoder_outputs = BaseModelOutput(
                last_hidden_state=encoder_outputs[0],
                hidden_states=encoder_outputs[1] if len(encoder_outputs) > 1 else None,
                attentions=encoder_outputs[2] if len(encoder_outputs) > 2 else None,
            )

        hidden_states = encoder_outputs[0]

        # Set device for model parallelism
        if self.model_parallel:
            torch.cuda.set_device(self.decoder.first_device)
            hidden_states = hidden_states.to(self.decoder.first_device)
            if decoder_input_ids is not None:
                decoder_input_ids = decoder_input_ids.to(self.decoder.first_device)
            if attention_mask is not None:
                attention_mask = attention_mask.to(self.decoder.first_device)
            if decoder_attention_mask is not None:
                decoder_attention_mask = decoder_attention_mask.to(self.decoder.first_device)

        # Decode
        decoder_outputs = self.decoder(
            input_ids=decoder_input_ids,
            attention_mask=decoder_attention_mask,
            inputs_embeds=decoder_inputs_embeds,
            past_key_values=past_key_values,
            encoder_hidden_states=hidden_states,
            encoder_attention_mask=attention_mask,
            head_mask=decoder_head_mask,
            cross_attn_head_mask=cross_attn_head_mask,
            use_cache=use_cache,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        if not return_dict:
            return decoder_outputs + encoder_outputs
        
        
        # Update Final Decoder Layer  to Zeto
        
        decoder_outputs.last_hidden_state=decoder_outputs.last_hidden_state.data.fill_(0.00) 

        return Seq2SeqModelOutput(
            last_hidden_state=decoder_outputs.last_hidden_state,
            past_key_values=decoder_outputs.past_key_values,
            decoder_hidden_states=decoder_outputs.hidden_states,
            decoder_attentions=decoder_outputs.attentions,
            cross_attentions=decoder_outputs.cross_attentions,
            encoder_last_hidden_state=encoder_outputs.last_hidden_state,
            encoder_hidden_states=encoder_outputs.hidden_states,
            encoder_attentions=encoder_outputs.attentions,
        )


### Class for Initialization Model 

In [83]:
T5Model = T5Model_Ov()

Some weights of the model checkpoint at google/flan-t5-small were not used when initializing T5Model: ['lm_head.weight']
- This IS expected if you are initializing T5Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


### Rerun the Question answering after intizing  T5_Model 

In [181]:
#tokenizer
input_text="question-answering: What is sum of one and one ?"
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

enc_outputs = T5Model.encoder(input_ids)
dec_outputs = T5Model.decoder(torch.tensor(enc_outputs.last_hidden_state.to(torch.int64)[0]))
print(tokenizer.decode(enc_outputs[0]))

un est un


### Loan Question Answering Train File

In [15]:



with open('topiocqa/topiocqa_train.json') as f:
    topiocqa_train_q_a = json.load(f)
    


### <font size=10>Fine Tunning Flan-T5-small for Question Answering Task</font>

### Load Pretrained Model

In [28]:
t5_model=AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small", return_dict=True)

### Parameter Tunning

In [ ]:
OPTIMIZER = Adam(t5_model.parameters(), lr=0.00001)
Q_LEN = 256   # Question Length
T_LEN = 32    # Target Length
BATCH_SIZE = 4
DEVICE = "cuda:0"

In [17]:
def prepare_data(data):
    q_c_a=[]
    for qca in data:
        inputs={"context":str(qca['Context']),"question":str(qca["Question"]),"answer":str(qca['Answer'])}
        q_c_a.append(inputs)
    return q_c_a

In [18]:
topiocqa_train_q_a_ctx=prepare_data(topiocqa_train_q_a)

In [21]:
topiocqa_train_q_a_ctx_df = pd.DataFrame(topiocqa_train_q_a_ctx)


In [138]:
topiocqa_train_q_a_ctx_df

,context,question,answer
0,[],what was australia's contribution to the battl...,The army personnel and thousands of Australian...
1,[what was australia's contribution to the batt...,was the battle fought in australia?,UNANSWERABLE
2,[what was australia's contribution to the batt...,when was the battle fought?,1944
3,[what was australia's contribution to the batt...,who fought in this battle?,Australians and British
4,[what was australia's contribution to the batt...,was this battle part of a bigger war?,UNANSWERABLE
...,...,...,...
45445,"[when does chloe find out about clarks powers,...",with regards to the actress who played sulliva...,Her first major television role was an episode...
45446,"[when does chloe find out about clarks powers,...",was she prosecuted on a criminal charge?,She was arrested in Brooklyn by the FBI on Apr...
45447,"[when does chloe find out about clarks powers,...",where were the scenes filmed during the making...,"BB Studios in Burnaby, British Columbia."
45448,"[when does chloe find out about clarks powers,...",who composed that television series' music score?,Mark Snow


### Preprocess the Data and do Tokenization

In [109]:
class QA_Dataset(Dataset):
    def __init__(self, tokenizer, dataframe, q_len, t_len):
        self.tokenizer = tokenizer
        self.q_len = q_len
        self.t_len = t_len
        self.data = dataframe
        self.questions = self.data["question"]
        self.context = self.data["context"]
        self.answer = self.data['answer']
        
    def __len__(self):
        return len(self.questions)
    
    def __getitem__(self, idx):
        question = self.questions[idx]
        context = self.context[idx]
        answer = self.answer[idx]
        
        question_tokenized = self.tokenizer(question, context, max_length=self.q_len, padding="max_length",
                                                    truncation=True, pad_to_max_length=True, add_special_tokens=True)
        answer_tokenized = self.tokenizer(answer, max_length=self.t_len, padding="max_length", 
                                          truncation=True, pad_to_max_length=True, add_special_tokens=True)
        
        labels = torch.tensor(answer_tokenized["input_ids"], dtype=torch.long)
        labels[labels == 0] = -100
        
        return {
            "input_ids": torch.tensor(question_tokenized["input_ids"], dtype=torch.long),
            "attention_mask": torch.tensor(question_tokenized["attention_mask"], dtype=torch.long),
            "labels": labels,
            "decoder_attention_mask": torch.tensor(answer_tokenized["attention_mask"], dtype=torch.long)
        }


### Train Test Split the Dataset

In [110]:
train_data, val_data = train_test_split(topiocqa_train_q_a_ctx_df, test_size=0.2, random_state=42)

train_sampler = RandomSampler(train_data.index)
val_sampler = RandomSampler(val_data.index)

qa_dataset = QA_Dataset(tokenizer, topiocqa_train_q_a_ctx_df, Q_LEN, T_LEN)

train_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=train_sampler)
val_loader = DataLoader(qa_dataset, batch_size=BATCH_SIZE, sampler=val_sampler)


### Training for 5 Epochs

In [111]:
t5_model=t5_model.to(DEVICE)

In [156]:
train_loss = 0
val_loss = 0
train_batch_count = 0
val_batch_count = 0

for epoch in range(5):
    t5_model.train()
    for batch in tqdm(train_loader, desc="Training batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = t5_model(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        train_loss += outputs.loss.item()
        train_batch_count += 1
    
    #Evaluation
    t5_model.eval()
    for batch in tqdm(val_loader, desc="Validation batches"):
        input_ids = batch["input_ids"].to(DEVICE)
        attention_mask = batch["attention_mask"].to(DEVICE)
        labels = batch["labels"].to(DEVICE)
        decoder_attention_mask = batch["decoder_attention_mask"].to(DEVICE)

        outputs = t5_model(
                          input_ids=input_ids,
                          attention_mask=attention_mask,
                          labels=labels,
                          decoder_attention_mask=decoder_attention_mask
                        )

        OPTIMIZER.zero_grad()
        outputs.loss.backward()
        OPTIMIZER.step()
        val_loss += outputs.loss.item()
        val_batch_count += 1
        
    print(f"{epoch+1}/{2} -> Train loss: {train_loss / train_batch_count}\tValidation loss: {val_loss/val_batch_count}")
    

Validation batches: 100%|███████████████████████████████████████████████████████████| 2273/2273 [03:50<00:00,  9.86it/s]


1/2 -> Train loss: 4.064034067404152	Validation loss: 3.7232743387855836


Validation batches: 100%|███████████████████████████████████████████████████████████| 2273/2273 [03:55<00:00,  9.65it/s]


2/2 -> Train loss: 4.065573071954798	Validation loss: 3.7244674818818915


Training batches:   4%|██▋                                                           | 392/9090 [00:41<15:14,  9.51it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Training batches:  98%|████████████████████████████████████████████████████████████ | 8948/9090 [15:47<00:15,  9.42it/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Training batches:  71%|███████████████████████████████████████████▏                 | 6427/9090 [11:21<04:41, 

5/2 -> Train loss: 4.065605407954574	Validation loss: 3.726745702503958


### Save the Model

In [157]:
t5_model.save_pretrained("q&a_model")

In [116]:
val_loader

In [139]:
val_data['question']

'name a few of the planets in the outer region?'

#### Check for Repsonce

In [143]:
input_text='name a few of the planets in the outer region?'
input_ids = tokenizer(input_text, return_tensors="pt").input_ids

outputs = t5_model.generate(input_ids.to(DEVICE))
print(tokenizer.decode(outputs[0]))

<pad> osrio</s>


###  Generate Prediction from Raw Text

In [158]:
predictions=[]
for q_iter in tqdm(val_data['question']):
    input_ids = tokenizer(q_iter, return_tensors="pt").input_ids
    outputs = t5_model.generate(input_ids.to(DEVICE))
    predictions.append(tokenizer.decode(outputs[0]))

100%|███████████████████████████████████████████████████████████████████████████████| 9090/9090 [14:51<00:00, 10.19it/s]


### Get the Metrices out from the model

In [159]:
rouge = evaluate.load('rouge')
#predictions=result
rouge_score=rouge.compute(predictions=predictions,references=val_data['answer'].to_list())
print(rouge_score)
bleu = evaluate.load("bleu")
bleu_score = bleu.compute(predictions=predictions, references=val_data['answer'].to_list())
print(bleu_score)

{'rouge1': 0.044862706740465254, 'rouge2': 0.0049390792052906585, 'rougeL': 0.04280564255114722, 'rougeLsum': 0.04281252560221893}
{'bleu': 0.0013019711131864536, 'precisions': [0.04047318327394109, 0.004334933279722565, 0.0005248524738992284, 0.0001139953750447839], 'brevity_penalty': 0.7233238467630417, 'length_ratio': 0.7553450655036713, 'translation_length': 88676, 'reference_length': 117398}
